### 학습 평가 데이터 만들기

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

In [2]:
train_dataset = dataset["train"].shuffle(seed=42).select([i for i in list(range(1000))])
validation_dataset = dataset["test"].shuffle(seed=42).select([i for i in list(range(100))])
test_dataset = dataset["test"].shuffle(seed=42).select([i for i in list(range(50))])


Loading cached shuffled indices for dataset at /Users/yangwoolee/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-a0e621c27d9b360e.arrow
Loading cached shuffled indices for dataset at /Users/yangwoolee/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-61e0da4d9cd46a2c.arrow
Loading cached shuffled indices for dataset at /Users/yangwoolee/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-61e0da4d9cd46a2c.arrow


In [5]:
train_dataset.to_csv('data/train.csv',index=False)
validation_dataset.to_csv('data/validation.csv',index=False)
test_dataset.to_csv('data/test.csv',index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

36382

In [6]:
from transformers import (
        DistilBertForSequenceClassification,
        DistilBertTokenizer,
        Trainer,
        TrainingArguments,
        TrainerCallback
    )
from datasets import Dataset
import pandas as pd
# loading data
train_path = 'data/train.csv'
evaluation_path = 'data/validation.csv'
train_dataset = Dataset.from_csv(train_path).select(range(8))
evaluation_dataset = Dataset.from_csv(evaluation_path)

Using custom data configuration default-791619f85ce504b1
/Users/yangwoolee/.pyenv/versions/3.9.1/lib/python3.9/site-packages/datasets/builder.py:712: FutureWarning: 'use_auth_token' was deprecated in version 2.7.1 and will be removed in 3.0.0. Pass `use_auth_token` to the initializer/`load_dataset_builder` instead.
  warnings.warn(
Found cached dataset csv (/Users/yangwoolee/.cache/huggingface/datasets/csv/default-791619f85ce504b1/0.0.0)
Using custom data configuration default-77a87d425e9ea9ea
Found cached dataset csv (/Users/yangwoolee/.cache/huggingface/datasets/csv/default-77a87d425e9ea9ea/0.0.0)


In [7]:
# tokenizing
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(item):
    return tokenizer(item["text"], padding="max_length", max_length=128, truncation=True)

train = train_dataset.map(tokenize_function)
evaluation = evaluation_dataset.map(tokenize_function)

print('complete Tokenizing')

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /Users/yangwoolee/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at /Users/yangwoolee/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/yangwoolee/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d7

complete Tokenizing


In [9]:
print(train[:1])

{'label': [4], 'text': ["I stalk this truck.  I've been to industrial parks where I pretend to be a tech worker standing in line, strip mall parking lots, and of course the farmer's market.  The bowls are so so absolutely divine.  The owner is super friendly and he makes each bowl by hand with an incredible amount of pride.  You gotta eat here guys!!!"], 'input_ids': [[101, 1045, 23899, 2023, 4744, 1012, 1045, 1005, 2310, 2042, 2000, 3919, 6328, 2073, 1045, 9811, 2000, 2022, 1037, 6627, 7309, 3061, 1999, 2240, 1010, 6167, 6670, 5581, 7167, 1010, 1998, 1997, 2607, 1996, 7500, 1005, 1055, 3006, 1012, 1996, 15220, 2024, 2061, 2061, 7078, 7746, 1012, 1996, 3954, 2003, 3565, 5379, 1998, 2002, 3084, 2169, 4605, 2011, 2192, 2007, 2019, 9788, 3815, 1997, 6620, 1012, 2017, 10657, 4521, 2182, 4364, 999, 999, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5
)



In [17]:
model.config._name_or_path

'distilbert-base-uncased'

In [ ]:

tra_arg = TrainingArguments(
    num_train_epochs=1,
    output_dir="test",
    logging_steps=10,
    # evaluation_strategy="epoch",
)

class myCallback(TrainerCallback):

    def on_log(self, args, state, control, logs=None, **kwargs):
        print(f'{state.global_step}회 진행 중 ')

trainer = Trainer(
    model=model,
    args=tra_arg,
    train_dataset=train,
    eval_dataset=evaluation,
    callbacks=[myCallback]
)

trainer.train()

In [5]:
tokenizer.save_pretrained('test123')

tokenizer config file saved in test123/tokenizer_config.json
Special tokens file saved in test123/special_tokens_map.json


('test123/tokenizer_config.json',
 'test123/special_tokens_map.json',
 'test123/vocab.txt',
 'test123/added_tokens.json')

### KFP를 활용해 Training PipeLine 만들기

In [1]:
#### New
from functools import partial
from kfp.components import create_component_from_func, InputPath, OutputPath



@partial(
    create_component_from_func,
    packages_to_install=["pandas"],
)
def load_data(
    train_path: OutputPath("csv"),
    evaluation_path: OutputPath("csv"),
):


    import pandas as pd

    # load data from github
    df_train = pd.read_csv(
        "https://raw.github.com/yangoos57/Learning_kubeflow/main/mini_project/data/train.csv"
    )
    df_evaluation = pd.read_csv(
        "https://raw.github.com/yangoos57/Learning_kubeflow/main/mini_project/data/validation.csv"
    )
    
    df_train.to_csv(train_path, index=False)
    df_evaluation.to_csv(evaluation_path, index=False)

    print('complete Loading Data')


In [4]:
@partial(
    create_component_from_func,
    base_image="679oose/basepython:1.0"
)
def train_model(
    train_path:InputPath("csv"),
    evaluation_path: InputPath("csv"),
    model_save_path: OutputPath("folder"),
):

    from transformers import (
        DistilBertForSequenceClassification,
        DistilBertTokenizer,
        Trainer,
        TrainingArguments,
        TrainerCallback
    )
    from datasets import Dataset

    # loading data
    train_dataset = Dataset.from_csv(train_path)
    evaluation_dataset = Dataset.from_csv(evaluation_path)

    # tokenizing
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_function(item):
        return tokenizer(item["text"], padding="max_length", max_length=128, truncation=True)

    train = train_dataset.map(tokenize_function)
    evaluation = evaluation_dataset.map(tokenize_function)

    print('complete Tokenizing')

    model = DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=len(set(train_dataset["label"]))
    )
    tra_arg = TrainingArguments(
        output_dir="test",
        num_train_epochs=1,
        logging_steps=5,
        evaluation_strategy="epoch",
        disable_tqdm=True,
        save_strategy = "no"
    )

    class myCallback(TrainerCallback):

        def on_log(self, args, state, control, logs=None, **kwargs):
            print(f'{state.global_step} Steps ')

    trainer = Trainer(
        model=model,
        args=tra_arg,
        train_dataset=train,
        eval_dataset=evaluation,
        callbacks=[myCallback]
    )

    trainer.train()
    trainer.save_model(model_save_path)



In [6]:
from kfp.dsl import pipeline

@pipeline(name="NLP_Pipeline")
def NLP_Pipeline():
    data = load_data()
    train_model(data.outputs['train'],data.outputs['evaluation'])


import kfp
if __name__ == "__main__":
    kfp.compiler.Compiler().compile(NLP_Pipeline, "NLP_Pipeline_1.2.yaml")